<center><h1>Mathematical Formulation</h1></center>

The economical dispach of thermal generation units can be formulated as an objetive function, which should be minimized considering the limits of each power plant and also the supply of the demanded power. The following system of equations formulate the problem:

$$
\begin{equation}
    \begin{aligned}
       \text{Min } & F_t = \sum_{i=1}^{n_g} F_i(P_i) \\
       \text{s.t}  & \\
                   & \sum_{i=1}^{n_g} P_i = P_D \\
                   & P_i^{min} \leq P_i \leq P_i^{max} \\
    \end{aligned}
\end{equation}
$$

Onde:
 - $F_t$ = total financial cost of the dispach.
 - $F_i$ = financial cost function for the the generator $i$.
 - $P_i$ = power generated by the $i$. 
 - $P_D$ = power demand.
 - $P_i^{min}$ = minimum generation limit of the generator $i$.
 - $P_i^{max}$ = maximum generation limit of the generator $i$;
 - $n_g$ = number of generatio units.

A quadratic function can be used as a proxy of the fuel cost for each generation unit (Zhu, 2015). 

$$
\begin{equation}
    F_i(P_i) = c_iP_i^2 + b_iP_i + a_i
\end{equation}
$$

Bringin a more realistic approach to the economic dispach, a thermal generation unit might also have operative zones (Orero e Irving, 1996), which are directly related to auxiliary systems such as boilers and feed pumps. Such operative zones should be avoided as they bring risk to a safe operation of the plant, hence a discontinuous mathematical formulation rises:

$$
\begin{equation}
        P_i^{min} \leq P_i \leq P_i^{n_1} \\
        P_i^{n_2} \leq P_i \leq P_i^{n_3} \\
        P_i^{n_4} \leq P_i \leq P_i^{max} \\
\end{equation}
$$

---

# Solver

In order to solve the problem the [CVXOPT](https://cvxopt.org/) library will be used. The library offers a complete set of mathematical solvers implemented.

### Loading `cvxopt`

In [4]:
from cvxopt import matrix, solvers

solvers.options['show_progress'] = False
# solvers.options['refinement'] = 2
solvers.options['maxiters'] = 15

### Loading data

From now on we can use the `systems` module and initialize a `PowerSystem`, which should provide all of the necessary data for the solver. 

In [1]:
from acopoweropt import system

PSystem = system.PowerSystem(name='s15')

### Initializing variables

In [2]:
operation = PSystem.rand_opt_zones()

df = PSystem.data.set_index('tgu')

In [18]:
import json
name = 'teste'
try:
    with open("systems.json") as f:
        content = json.loads(f.read())
        psystem = content[name]
except:
    raise Exception("Error reading system '{}' from systems.json".format(name))

Exception: Error reading system 'teste' from systems.json

In [6]:
psystem

In [3]:
def sample_operation(power_system_data: pd.DataFrame) -> pd.DataFrame:
    l = []
    for tgu in power_system_data.index.unique():
        possible_operations = power_system_data.loc[tgu]

        if type(possible_operations) == pd.DataFrame:
            # Randomly sample one option if more than one is available
            l.append(possible_operations.sample())
        else:
            # Restructure the operations back to a dataframe to be composed
            l.append(possible_operations.to_frame()
                                        .transpose()
                                        .rename_axis('tgu'))
            
    return pd.concat(l)

NameError: name 'pd' is not defined

In [142]:
sample_operation(power_system_data=df)

,a,b,c,Pmin,Pmax
tgu,,,,,
1,671.03,10.07,0.000299,150,455
2,574.54,10.22,0.000183,225,305
3,374.59,8.8,0.001126,20,130
4,374.59,8.8,0.001126,20,130
5,461.37,10.4,0.000205,335,390
6,630.14,10.1,0.000301,455,460
7,548.2,9.87,0.000364,135,465
8,227.09,11.5,0.000338,60,300
9,173.72,11.21,0.000807,25,162


In [94]:
possible_operations = df.loc[tgu].to_frame().transpose().rename_axis('tgu')

,a,b,c,Pmin,Pmax
tgu,,,,,
1,671.03,10.07,0.000299,150,455


In [98]:
df.loc[tgu]

,a,b,c,Pmin,Pmax
tgu,,,,,
2,574.54,10.22,0.000183,150,185
2,574.54,10.22,0.000183,225,305
2,574.54,10.22,0.000183,335,420
2,574.54,10.22,0.000183,450,455


In [58]:
type(df.loc[2]) == pd.DataFrame

True

In [57]:
import pandas as pd
pd.DataFrame

type